In [1]:
import re
import sqlite3
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, LSTM, Dense, Dropout, Activation
from keras.callbacks import EarlyStopping, TensorBoard, LambdaCallback
from keras.models import Sequential
import keras.utils as ku 
import numpy as np 


""" Custom Libs """
import Cleaner as c
from embedding import ngram, generate_text_ngram

Using TensorFlow backend.


In [0]:
#Hyperparameters

epochs = 150
hidden_1 = 200
hidden_2 = 150
embedding_layer = 64
dropout_1 = 0.1
seed = 0 #For reproducibility

np.random.seed(seed)

In [0]:
#Import Data
sqlite_file = 'deeplearning.sqlite'
table_name  = 'tweets'
cd          = c.CleanData(sqlite_file, table_name)
q           ='SELECT * FROM {} WHERE Author == "simonlporter";'.format(table_name)

cd.set_table(q)
raw_data = cd.get_clean_table()
raw_data = raw_data.CleanText.values
data = ''
for x in raw_data:
    data += x + "\n"
    
sentences = [x.split(' ') for x in raw_data]



In [0]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(sentences, test_size = 0.2)
comments = train + test 

In [5]:
ls_x = []
ls_y = []

for x in range(len(comments)):
  for y in range(len(comments[x])):
    if len(comments[x][y]) > 18:
      error = comments[x][y]
      print(error, x, y)
      ls_x.append(x)
      ls_y.append(y)

      
      
      

enterprisearchitecture 334 1
forrestergaggleamputmcampaignexternal20gaggleamputmcontenthttpsxeroxbz2aspggn1660116 492 6


In [0]:
error_1 = comments[ls_x[0]][ls_y[0]]
error_2 = comments[ls_x[1]][ls_y[1]]
for comment in comments:
  if error_1 in comment:
    comment.remove(error_1)
  if error_2 in comment:
    comment.remove(error_2)

In [0]:
#Word2vec
from gensim.models import Word2Vec
window = 30 #no of context
min_count = 1 #min number of times word has to appear to be included
sg = 0 #skip gram = 1, cbow = 0
iteration = 200 #epochs
word_model = Word2Vec(comments,size = 200, window = window, min_count = min_count, sg = sg, iter = iteration) 
pretrained_weights = word_model.wv.vectors
vocab_size, embedding_size = pretrained_weights.shape

In [8]:
print("Number of word vectors: {}".format(len(word_model.wv.vocab)))
word_model.wv.vocab

Number of word vectors: 1078


{'how': <gensim.models.keyedvectors.Vocab at 0x7f270608e8d0>,
 'data': <gensim.models.keyedvectors.Vocab at 0x7f270608e978>,
 'analytics': <gensim.models.keyedvectors.Vocab at 0x7f270608e6d8>,
 'has': <gensim.models.keyedvectors.Vocab at 0x7f270608e668>,
 'changed': <gensim.models.keyedvectors.Vocab at 0x7f270608eb00>,
 'or': <gensim.models.keyedvectors.Vocab at 0x7f270608eac8>,
 'not': <gensim.models.keyedvectors.Vocab at 0x7f270608eb38>,
 'over': <gensim.models.keyedvectors.Vocab at 0x7f270608eba8>,
 'the': <gensim.models.keyedvectors.Vocab at 0x7f270608eb70>,
 'last': <gensim.models.keyedvectors.Vocab at 0x7f270608e400>,
 '30': <gensim.models.keyedvectors.Vocab at 0x7f270608e630>,
 'years': <gensim.models.keyedvectors.Vocab at 0x7f270608e438>,
 'digging': <gensim.models.keyedvectors.Vocab at 0x7f270608e4a8>,
 'deeper': <gensim.models.keyedvectors.Vocab at 0x7f270608e5c0>,
 'into': <gensim.models.keyedvectors.Vocab at 0x7f270608e5f8>,
 'ai': <gensim.models.keyedvectors.Vocab at 0x7f2

In [9]:
word_model.wv.most_similar_cosmul(positive=['deep', 'learning'], ) #Test

[('reinforcement', 0.9052487015724182),
 ('difference', 0.8863771557807922),
 ('whats', 0.8742845058441162),
 ('explanation', 0.8181771636009216),
 ('between', 0.805482029914856),
 ('practical', 0.7480459213256836),
 ('intelligence', 0.7364580035209656),
 ('artificial', 0.7321343421936035),
 ('continuous', 0.7108794450759888),
 ('machine', 0.7096708416938782)]

In [0]:
def word2idx(word):
  return word_model.wv.vocab[word].index
def idx2word(idx):
  return word_model.wv.index2word[idx]

In [11]:
print('\nPreparing the data for LSTM...')
max_sentence_len = max([len(sentence) for sentence in sentences])

train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)


Preparing the data for LSTM...
train_x shape: (499, 25)
train_y shape: (499,)


In [12]:
print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights]))
model.add(LSTM(300, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(200))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


Training LSTM...


In [0]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [0]:
def generate_next(text, num_generated=10):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs))
    idx = sample(prediction[-1], temperature=1)
    word_idxs.append(idx)
    if idx2word(idx) == '.':
        return ' '.join(idx2word(idx) for idx in word_idxs)
  return ' '.join(idx2word(idx) for idx in word_idxs)

In [0]:
def on_epoch_end(epoch, _):
  print('\nGenerating text after epoch: %d' % epoch)
  texts = [
    'data',
    'deep',
    'machine',
  ]
  for text in texts:
    sample = generate_next(text)
    print('%s... -> %s' % (text, sample))

In [91]:
model.fit(train_x, train_y,
          epochs=200,
          #callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)],
         verbose = 2)

Epoch 1/200
 - 2s - loss: 1.3530e-04
Epoch 2/200
 - 2s - loss: 1.3515e-04
Epoch 3/200
 - 2s - loss: 1.3472e-04
Epoch 4/200
 - 2s - loss: 1.3452e-04
Epoch 5/200
 - 2s - loss: 1.3427e-04
Epoch 6/200
 - 2s - loss: 1.3405e-04
Epoch 7/200
 - 2s - loss: 1.3379e-04
Epoch 8/200
 - 2s - loss: 1.3375e-04
Epoch 9/200
 - 2s - loss: 1.3338e-04
Epoch 10/200
 - 2s - loss: 1.3330e-04
Epoch 11/200
 - 2s - loss: 1.3297e-04
Epoch 12/200
 - 2s - loss: 1.3277e-04
Epoch 13/200
 - 2s - loss: 1.3262e-04
Epoch 14/200
 - 2s - loss: 1.3230e-04
Epoch 15/200
 - 2s - loss: 1.3215e-04
Epoch 16/200
 - 2s - loss: 1.3191e-04
Epoch 17/200
 - 2s - loss: 1.3165e-04
Epoch 18/200
 - 2s - loss: 1.3162e-04
Epoch 19/200
 - 2s - loss: 1.3119e-04
Epoch 20/200
 - 2s - loss: 1.3110e-04
Epoch 21/200
 - 2s - loss: 1.3082e-04
Epoch 22/200
 - 2s - loss: 1.3063e-04
Epoch 23/200
 - 2s - loss: 1.3032e-04
Epoch 24/200
 - 2s - loss: 1.3025e-04
Epoch 25/200
 - 2s - loss: 1.2996e-04
Epoch 26/200
 - 2s - loss: 1.2981e-04
Epoch 27/200
 - 2s - 

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 200)         215600    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 300)         601200    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               400800    
_________________________________________________________________
dense_1 (Dense)              (None, 1078)              216678    
_________________________________________________________________
activation_1 (Activation)    (None, 1078)              0         
Total params: 1,434,278
Trainable params: 1,434,278
Non-trainable params: 0
_________________________________________________________________


In [89]:
generate_next("deep learning", 10)

'deep learning managed center leave demystifying nearly joined ipo rise balance worsen'

In [84]:
generate_next("machine learning", 10)

'machine learning chart power succession selfaware planning products, loss heres when people'

In [85]:
generate_next("This", 10)

'this bias worsen stressed years makes mcafee throwing do alone manage'

In [73]:
generate_next("The", 10)

'the growth securing innovation, strategic each metadata hours steps services con'

In [0]:
def rand_word():
  return list(word_model.wv.vocab)[np.random.randint(0, len(word_model.wv.vocab))]

In [51]:
generate_next(rand_word(), 20)

'when by crazy big suggests corporate opposite worklife securing prefer if strategy hard top case ship lose chart teaching its are'

In [42]:
generate_next(rand_word(), 20)

'damage dont how reimagining transformations up, ambitious industry between challenges workday context reduce way doctor lack body look theres include outcomes'

In [43]:
generate_next(rand_word(), 20)

'ai robotic prepare adapt foundational influencers elements bffs heart doesnt ground explained personalities. today 21 five going money remote service worker'

In [44]:
generate_next(rand_word(), 20)


'tools changed remain architect beyond results insights revenue top focus uniper, inclusion korn followers trust wrong ready an pervasive house mind'